In [32]:
import pandas as pd


In [ ]:
item_categories = pd.read_csv('data/item_categories.csv')
item_daily_features = pd.read_csv('data/item_daily_features.csv')
kuairec_caption = pd.read_csv('data/kuairec_caption_category.csv', encoding='utf-8',
    na_values=[],
    keep_default_na=False,
    on_bad_lines='skip',   # skip problematic rows
    engine='python')

In [34]:
print(item_categories.feat)

0            [8]
1        [27, 9]
2            [9]
3           [26]
4            [5]
          ...   
10723       [11]
10724        [2]
10725       [15]
10726       [19]
10727        [5]
Name: feat, Length: 10728, dtype: object


In [35]:
import ast

metadata = item_categories.copy()

# add in metadata the videos categoories
for i in range (31):
    metadata['category_' + str(i)] = 0

metadata['feat'] = metadata['feat'].apply(ast.literal_eval)
for index, row in metadata.iterrows():
    # Get the list of features for the current row
    features = row['feat']

    # Set the corresponding category columns to 1
    for feat in features:
        col_name = f'category_{feat}'
        if col_name in metadata.columns:
            metadata.at[index, col_name] = 1

metadata.head()

# add in metadata the videos metadata in item_daily_features merge on video_id
video_metadata = item_daily_features[['video_id', 'author_id', 'video_type', 'upload_dt', 'video_duration', 'music_id']]
# I only want one row per video_id
video_metadata = video_metadata.drop_duplicates(subset=['video_id'])
print(len(video_metadata))

# Convert video type to int meaning if it is AD it is 0 and if it is NORMAL 1
video_metadata['video_type'] = video_metadata['video_type'].apply(lambda x: 0 if x == 'AD' else 1)
metadata['video_type'] = video_metadata['video_type']
video_metadata.drop(columns=['video_type'], inplace=True)
metadata = metadata.merge(video_metadata, how='inner', left_on='video_id', right_on='video_id')
metadata.drop(columns=['feat'], inplace=True)

# convert upload_dt to datetime
metadata['upload_dt'] = pd.to_datetime(metadata['upload_dt'], format='%Y-%m-%d')
# convert datetime to unix timestamp
metadata['upload_dt'] = metadata['upload_dt'].astype('int64') // 10**9
metadata.dtypes

10728


video_id            int64
category_0          int64
category_1          int64
category_2          int64
category_3          int64
category_4          int64
category_5          int64
category_6          int64
category_7          int64
category_8          int64
category_9          int64
category_10         int64
category_11         int64
category_12         int64
category_13         int64
category_14         int64
category_15         int64
category_16         int64
category_17         int64
category_18         int64
category_19         int64
category_20         int64
category_21         int64
category_22         int64
category_23         int64
category_24         int64
category_25         int64
category_26         int64
category_27         int64
category_28         int64
category_29         int64
category_30         int64
video_type        float64
author_id           int64
upload_dt           int64
video_duration    float64
music_id            int64
dtype: object

In [57]:
# We will consider the watch ratio as a rating
interactions = pd.read_csv('data/small_matrix.csv')
interactions.drop(columns=['play_duration', 'video_duration', 'date', 'timestamp'], inplace=True)
# convert the watch ratio to have a float with 2 decimal points
interactions['watch_ratio'] = interactions['watch_ratio'].apply(lambda x: round(x, 2))
interactions.dropna(inplace=True)

In [58]:
from sklearn.model_selection import train_test_split
df_interaction_train, df_interaction_test = train_test_split(interactions, test_size=0.05, random_state=42)

In [59]:
len(df_interaction_train), len(df_interaction_test)

(4269849, 224729)

In [60]:
# If needed filter to take only video with a certain number of interactions

In [61]:
df_interaction_train

,user_id,video_id,time,watch_ratio
3629783,5582,2305,2020-07-19 19:21:08.754,0.48
2501229,3935,2847,2020-08-06 19:09:15.721,0.91
2865248,4505,2476,2020-08-03 15:04:15.377,1.21
809873,1308,9572,2020-07-25 02:45:51.74,1.42
1413096,2205,3846,2020-07-24 01:39:23.707,1.08
...,...,...,...,...
1181323,1853,726,2020-07-27 10:53:33.699,1.37
1633180,2530,1301,2020-08-12 18:02:09.223,0.89
2324232,3660,3964,2020-07-18 07:30:35.159,0.42
4479227,6850,5826,2020-08-02 02:18:33.531,0.80


In [62]:
# Create a user-item matrix
df_interaction_train = df_interaction_train.pivot(index='user_id', columns='video_id', values='watch_ratio').fillna(0)
df_interaction_train.head()
# Fill NaN values with -1
df_interaction_train = df_interaction_train.fillna(-1)

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

metadata.dropna(inplace=True)
cosine_matrix = cosine_similarity(metadata)

In [66]:
# Recommend the top 5 videos for a given user
def recommend_videos(user_id, num_recommendations=5):
    # Get the user's watched videos
    user_watched_videos = df_interaction_train.loc[user_id]
    # Get the indices of the videos that the user has not watched
    unwatched_video_indices = user_watched_videos[user_watched_videos == -1].index
    # Calculate the similarity scores for the unwatched videos
    similarity_scores = cosine_matrix[user_watched_videos.index][unwatched_video_indices]
    # Get the top N recommendations
    top_video_indices = similarity_scores.argsort()[-num_recommendations:][::-1]
    return metadata.iloc[top_video_indices]

In [67]:
recommend_videos(5582, num_recommendations=5)

IndexError: index 183 is out of bounds for axis 0 with size 183